In [1]:
import pandas as pd
from collections import Counter
from conllu import parse
import matplotlib.pyplot as plt
from collections import defaultdict
import seaborn as sns
from scipy.stats import chi2_contingency
import numpy as np
from scipy.stats import ks_2samp
from scipy.stats import ttest_ind
from scipy import stats  # Adicionar esta linha
from scipy.spatial.distance import jensenshannon  # Add this import
from matplotlib.ticker import MultipleLocator  # Adicionar esta linha


In [2]:
def parsear_conllu(caminho_arquivo):
    with open(caminho_arquivo, 'r', encoding='utf-8') as file:
        # Lê todo o conteúdo do arquivo de uma vez
        conteudo = file.read()
    
    try:
        # Parse todo o conteúdo como um único bloco
        dados = parse(conteudo)
        
        # Debug para verificar a estrutura
        if len(dados) > 0:
            primeira_sentenca = dados[0]
            print(f"\nDebug do parsing de {caminho_arquivo}:")
            print(f"Total de sentenças: {len(dados)}")
            print(f"Primeira sentença tem {len(primeira_sentenca)} tokens")
            print("Primeiros 3 tokens da primeira sentença:")
            for i, token in enumerate(primeira_sentenca[:3]):
                print(f"Token {i+1}: {dict(token)}")
        
        return dados
        
    except Exception as e:
        print(f"Erro ao fazer parsing do arquivo {caminho_arquivo}")
        print(f"Detalhes do erro: {e}")
        raise

# Exemplo de uso
caminho_arquivo1 = r'C:\Users\Marcos\Desktop\tic\bases\DANTEStocksV2.1.conllu'
caminho_arquivo2 = r'C:\Users\Marcos\Desktop\tic\bases\DANTEStocksV2.1_reconstruido.conllu'
caminho_arquivo3 = r'DANTES_Large_treated.conllu'

# Parseia o conteúdo dos arquivos
dados_conllu1 = parsear_conllu(caminho_arquivo1)
dados_conllu2 = parsear_conllu(caminho_arquivo2)
dados_conllu3 = parsear_conllu(caminho_arquivo3)



Debug do parsing de C:\Users\Marcos\Desktop\tic\bases\DANTEStocksV2.1.conllu:
Total de sentenças: 4042
Primeira sentença tem 22 tokens
Primeiros 3 tokens da primeira sentença:
Token 1: {'id': 1, 'form': 'Notas', 'lemma': 'nota', 'upos': 'NOUN', 'xpos': None, 'feats': {'Gender': 'Fem', 'Number': 'Plur'}, 'head': 10, 'deprel': 'parataxis', 'deps': None, 'misc': None}
Token 2: {'id': 2, 'form': 'gerais', 'lemma': 'geral', 'upos': 'ADJ', 'xpos': None, 'feats': {'Number': 'Plur'}, 'head': 1, 'deprel': 'amod', 'deps': None, 'misc': None}
Token 3: {'id': 3, 'form': 'A', 'lemma': 'o', 'upos': 'DET', 'xpos': None, 'feats': {'Definite': 'Def', 'Gender': 'Fem', 'Number': 'Sing', 'PronType': 'Art'}, 'head': 4, 'deprel': 'det', 'deps': None, 'misc': None}

Debug do parsing de C:\Users\Marcos\Desktop\tic\bases\DANTEStocksV2.1_reconstruido.conllu:
Total de sentenças: 4042
Primeira sentença tem 22 tokens
Primeiros 3 tokens da primeira sentença:
Token 1: {'id': 1, 'form': 'Notas', 'lemma': '_', 'upos'

In [3]:
# Adicionar no início do arquivo, após os imports
Dantes_Automatizado = "DanteStocks Automatizado"
Dantes_Large = "DanteStocks Large"
Dantes_Manual = "DanteStocks Manual"


In [4]:
def analise_tags_por_sentenca(corpora_data, nomes_corpora, caminho_grafico_tags, arquivo_latex):
    """
    Análise de tags por sentença para múltiplos corpora (até 4)
    """
    if len(corpora_data) > 4:
        raise ValueError("Esta função suporta no máximo 4 corpora")
    
    if len(corpora_data) != len(nomes_corpora):
        raise ValueError("O número de corpora deve ser igual ao número de nomes")
    
    # Cores para cada corpus
    cores = ['skyblue', 'lightcoral', 'lightgreen', 'plum']
    
    # Coletar número de tags por bloco de texto para cada corpus
    tags_por_bloco = []
    stats = []
    for corpus in corpora_data:
        tags = [len(sentence) for sentence in corpus]
        tags_por_bloco.append(tags)
        stats.append({
            'média': np.mean(tags),
            'mediana': np.median(tags),
            'desvio': np.std(tags),
            'q1': np.percentile(tags, 25),
            'q3': np.percentile(tags, 75),
            'total_sentencas': len(tags)
        })
    
    # Criar boxplots individuais
    for i, (tags, nome, cor, stat) in enumerate(zip(tags_por_bloco, nomes_corpora, cores, stats)):
        plt.figure(figsize=(10, 6))
        
        bp = plt.boxplot(tags, patch_artist=True,
                        boxprops=dict(facecolor=cor, alpha=0.7),
                        medianprops=dict(color='red', linewidth=2),
                        meanprops=dict(marker='D', markerfacecolor='darkblue', markersize=8),
                        flierprops=dict(marker='o', markerfacecolor='gray', markersize=4, alpha=0.5),
                        showmeans=True)
        
        plt.grid(True, axis='y', linestyle='--', alpha=0.7)
        plt.title(f'Distribuição - {nome}')
        plt.ylabel('Número de Tags por Sentença')
        
        # Adicionar anotações com estatísticas
        plt.text(0.95, 0.95, 
                f'Total de Sentenças: {stat["total_sentencas"]:,}\n'
                f'Média: {stat["média"]:.1f}\nMediana: {stat["mediana"]:.1f}\n'
                f'Desvio: {stat["desvio"]:.1f}\n'
                f'Q1: {stat["q1"]:.1f}\nQ3: {stat["q3"]:.1f}',
                transform=plt.gca().transAxes, 
                verticalalignment='top',
                horizontalalignment='right',
                bbox=dict(facecolor='white', alpha=0.8))
        
        plt.tight_layout()
        plt.savefig(f"{caminho_grafico_tags}_{i+1}.png", dpi=300, bbox_inches='tight')
        plt.close()
    
    # Criar gráfico comparativo
    plt.figure(figsize=(10, 6))
    
    # Preparar dados para o gráfico comparativo
    metricas = ['Média', 'Mediana', 'Desvio']
    valores = np.array([[s['média'], s['mediana'], s['desvio']] for s in stats])
    
    # Criar gráfico de barras agrupadas
    x = np.arange(len(metricas))
    width = 0.8 / len(corpora_data)
    
    for i, (nome, cor, vals) in enumerate(zip(nomes_corpora, cores, valores)):
        pos = x + width * (i - (len(corpora_data)-1)/2)
        bars = plt.bar(pos, vals, width, label=nome, color=cor, alpha=0.7)
        
        # Adicionar valores em cima das barras
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.1f}',
                    ha='center', va='bottom')
    
    plt.ylabel('Valor')
    plt.title('Comparação de Estatísticas')
    plt.xticks(x, metricas)
    plt.legend()
    plt.grid(True, axis='y', linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.savefig(f"{caminho_grafico_tags}_comparativo.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # Gerar relatório LaTeX
    arquivo_latex.write(
        "\\section{Análise de Tags por Sentença}\n"
        "Esta seção apresenta uma análise da distribuição do número de tags por sentença "
        "em cada corpus, permitindo comparar a complexidade estrutural das sentenças.\n\n"
    )
    
    # Tabela de estatísticas
    arquivo_latex.write("\\begin{table}[H]\n\\centering\n")
    arquivo_latex.write("\\begin{tabular}{|l|" + "c|"*len(corpora_data) + "}\\hline\n")
    arquivo_latex.write("Métrica & " + " & ".join(nomes_corpora) + " \\\\ \\hline\n")
    
    metricas = [
        ('Total de Sentenças', 'total_sentencas', '{:,}'),
        ('Média', 'média', '{:.2f}'),
        ('Mediana', 'mediana', '{:.2f}'),
        ('Desvio Padrão', 'desvio', '{:.2f}'),
        ('Primeiro Quartil (Q1)', 'q1', '{:.2f}'),
        ('Terceiro Quartil (Q3)', 'q3', '{:.2f}')
    ]
    
    for nome_metrica, chave, formato in metricas:
        valores = [formato.format(stat[chave]) for stat in stats]
        arquivo_latex.write(f"{nome_metrica} & {' & '.join(valores)} \\\\ \\hline\n")
    
    arquivo_latex.write(
        "\\end{tabular}\n"
        "\\caption{Estatísticas do Número de Tags por Sentença}\n"
        "\\end{table}\n\n"
    )
    
    # Inserir os quatro gráficos
    for i, nome in enumerate(nomes_corpora):
        arquivo_latex.write(salvar_grafico_e_gerar_latex(
            f"{caminho_grafico_tags}_{i+1}.png",
            f"Distribuição do Número de Tags por Sentença - {nome}"
        ))
    
    arquivo_latex.write(salvar_grafico_e_gerar_latex(
        f"{caminho_grafico_tags}_comparativo.png",
        "Comparação de Estatísticas entre os Corpora"
    ))
    
    arquivo_latex.write("\\clearpage\n")
    
    return {
        'estatisticas': stats,
        'tags_por_bloco': tags_por_bloco
    }

In [5]:
def analise_jensen_shannon(corpora_data, nomes_corpora, caminho_grafico, arquivo_latex):
    """
    Análise de distribuições usando Jensen-Shannon distance para múltiplos corpora (até 4)
    
    Args:
        corpora_data: Lista de corpora para análise
        nomes_corpora: Lista com os nomes dos corpora
        caminho_grafico: Caminho para salvar o gráfico
        arquivo_latex: Arquivo para escrita do relatório LaTeX
    """
    if len(corpora_data) > 4:
        raise ValueError("Esta função suporta no máximo 4 corpora")
    
    if len(corpora_data) != len(nomes_corpora):
        raise ValueError("O número de corpora deve ser igual ao número de nomes")
    
    # Cores para cada corpus
    cores = ['lightblue', 'lightcoral', 'lightgreen', 'plum']
    
    # Calcular frequências relativas para cada corpus
    pos_counters = []
    totals = []
    for corpus in corpora_data:
        counter = Counter(token['upos'] for sentence in corpus for token in sentence if token['upos'])
        pos_counters.append(counter)
        totals.append(sum(counter.values()))
    
    # Obter todas as tags únicas
    pos_tags = sorted(set().union(*[set(counter.keys()) for counter in pos_counters]))
    
    # Calcular distribuições de probabilidade
    distributions = []
    for counter, total in zip(pos_counters, totals):
        freq_rel = {tag: counter.get(tag, 0)/total for tag in pos_tags}
        distributions.append(np.array([freq_rel[tag] for tag in pos_tags]))
    
    # Calcular distâncias JS entre todos os pares
    js_results = []
    for i in range(len(distributions)):
        for j in range(i+1, len(distributions)):
            js_distance = jensenshannon(distributions[i], distributions[j])
            js_divergence = js_distance ** 2
            js_results.append({
                'corpus1': nomes_corpora[i],
                'corpus2': nomes_corpora[j],
                'distance': js_distance,
                'divergence': js_divergence
            })
    
    # Criar visualização
    plt.figure(figsize=(15, 8))
    
    # Gráfico de barras lado a lado
    x = np.arange(len(pos_tags))
    width = 0.8 / len(distributions)  # Ajusta largura baseado no número de corpora
    
    for i, (dist, nome, cor) in enumerate(zip(distributions, nomes_corpora, cores)):
        position = x + width * (i - (len(distributions)-1)/2)
        plt.bar(position, dist * 100, width, label=nome, color=cor, alpha=0.8)
    
    plt.grid(True, axis='y', linestyle='--', alpha=0.7)
    plt.title('Distribuição de Classes Gramaticais e Divergência de Jensen-Shannon', fontsize=14, pad=20)
    plt.xlabel('Classes Gramaticais', fontsize=12)
    plt.ylabel('Porcentagem (%)', fontsize=12)
    plt.xticks(x, pos_tags, rotation=45, ha='right')
    plt.legend()
    
    # Adicionar anotação com os valores das divergências
    js_text = "Distâncias JS:\n"
    for result in js_results:
        js_text += f"{result['corpus1']} vs {result['corpus2']}:\n"
        js_text += f"Dist = {result['distance']:.4f}\n"
        js_text += f"Div = {result['divergence']:.4f}\n"
    
    plt.text(0.98, 0.98, js_text,
             transform=plt.gca().transAxes, ha='right', va='top',
             bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))
    
    plt.tight_layout()
    plt.savefig(caminho_grafico, dpi=300, bbox_inches='tight')
    plt.close()
    
    # Gerar relatório LaTeX
    arquivo_latex.write(
        "\\section{Análise de Similaridade entre Distribuições}\n\n"
        
        "\\subsection{Introdução à Análise}\n"
        "Esta seção apresenta uma análise comparativa detalhada entre os corpora, utilizando métricas "
        "estatísticas avançadas para quantificar suas semelhanças e diferenças. O foco principal está na "
        "distribuição das Etiquetas POS (Part of Speech) em cada corpus.\n\n"
        
        "\\subsection{Metodologia: Distância de Jensen-Shannon}\n"
        "A análise utiliza a Distância de Jensen-Shannon (JS), uma métrica estatística sofisticada que:\n"
        "\\begin{itemize}\n"
        "\\item Mede o grau de similaridade entre duas distribuições de probabilidade\n"
        "\\item Produz valores entre 0 e 1, onde:\n"
        "  \\begin{itemize}\n"
        "  \\item 0 indica distribuições idênticas\n"
        "  \\item 1 indica distribuições completamente diferentes\n"
        "  \\end{itemize}\n"
        "\\item É mais robusta que medidas simples como diferença percentual\n"
        "\\end{itemize}\n\n"
        
        "\\subsection{Resultados das Comparações}\n"
    )
    
    # Adicionar tabela de resultados
    arquivo_latex.write(
        "\\begin{table}[H]\n\\centering\n"
        "\\begin{tabular}{|c|c|c|c|}\n\\hline\n"
        "\\textbf{Corpus 1} & \\textbf{Corpus 2} & \\textbf{Distância JS} & \\textbf{Divergência JS} \\\\ \\hline\n"
    )
    
    for result in js_results:
        arquivo_latex.write(
            f"{result['corpus1']} & {result['corpus2']} & "
            f"{result['distance']:.4f} & {result['divergence']:.4f} \\\\ \\hline\n"
        )
    
    arquivo_latex.write("\\end{tabular}\n\\caption{Resultados da Análise de Jensen-Shannon}\n\\end{table}\n\n")
    
    # Adicionar interpretações para cada par
    arquivo_latex.write("\\subsection{Interpretação dos Resultados}\n")
    for result in js_results:
        if result['distance'] < 0.1:
            interpretacao = "muito similares, sugerindo uma forte consistência no uso de classes gramaticais"
        elif result['distance'] < 0.2:
            interpretacao = "moderadamente similares, com algumas variações sutis no uso de classes gramaticais"
        elif result['distance'] < 0.3:
            interpretacao = "moderadamente diferentes, indicando variações significativas no uso de classes gramaticais"
        else:
            interpretacao = "substancialmente diferentes, demonstrando padrões distintos no uso de classes gramaticais"
        
        arquivo_latex.write(
            f"\\textbf{{{result['corpus1']} vs {result['corpus2']}:}} Os corpora são {interpretacao}.\n\n"
        )
    
    arquivo_latex.write(
        "\\subsection{Visualização dos Resultados}\n"
        "O gráfico abaixo apresenta uma comparação visual das distribuições de classes gramaticais "
        "entre todos os corpora analisados.\n\n"
    )
    
    arquivo_latex.write(salvar_grafico_e_gerar_latex(caminho_grafico, 
        "Distribuição de Classes Gramaticais e Divergência de Jensen-Shannon"))
    
    arquivo_latex.write("\\clearpage\n")

    return js_results

In [6]:
def teste_ks(corpora_data, nomes_corpora, caminho_grafico, arquivo_latex):
    """
    Realiza o teste Kolmogorov-Smirnov para comparar as distribuições de POS tags entre múltiplos corpora.
    """
    if len(corpora_data) > 4:
        raise ValueError("Esta função suporta no máximo 4 corpora")
    
    if len(corpora_data) != len(nomes_corpora):
        raise ValueError("O número de corpora deve ser igual ao número de nomes")
    
    # Cores para cada corpus
    cores = ['coral', 'turquoise', 'lightgreen', 'plum']
    
    # Contagem de POS tags para cada corpus
    def contar_pos_tags(corpus):
        contador = Counter()
        total_tokens = 0
        for sentence in corpus:
            for token in sentence:
                if token['upostag']:
                    contador[token['upostag']] += 1
                    total_tokens += 1
        return contador, total_tokens

    # Coletar dados para todos os corpora
    pos_counters = []
    totals = []
    for corpus in corpora_data:
        counter, total = contar_pos_tags(corpus)
        pos_counters.append(counter)
        totals.append(total)
    
    # Obter todas as tags únicas e ordenadas
    pos_tags = sorted(set().union(*[set(counter.keys()) for counter in pos_counters]))
    
    # Configurar estilo seaborn
    sns.set_style("whitegrid")
    sns.set_palette("Set2")
    plt.rcParams['font.size'] = 12
    
    # Calcular frequências normalizadas para cada corpus
    frequencies = []
    cum_frequencies = []
    for counter, total in zip(pos_counters, totals):
        freq = np.array([counter.get(pos, 0)/total for pos in pos_tags])
        frequencies.append(freq)
        cum_frequencies.append(np.cumsum(freq) * 100)
    
    # População cumulativa
    cum_pop = np.linspace(0, 100, len(pos_tags))
    
    # Realizar testes KS entre todos os pares
    ks_results = []
    for i in range(len(frequencies)):
        for j in range(i+1, len(frequencies)):
            estatistica, p_valor = stats.ks_2samp(frequencies[i], frequencies[j])
            ks_results.append({
                'corpus1': nomes_corpora[i],
                'corpus2': nomes_corpora[j],
                'estatistica': estatistica,
                'p_valor': p_valor,
                'cum_freq1': cum_frequencies[i],
                'cum_freq2': cum_frequencies[j],
                'cor1': cores[i],
                'cor2': cores[j]
            })
    
    # Criar figura para cada comparação
    for idx, result in enumerate(ks_results):
        fig, ax = plt.subplots(figsize=(14, 10))
        
        # Plotar linhas cumulativas
        ax.plot(cum_pop, result['cum_freq1'], 
               '-o', color=result['cor1'], label=result['corpus1'],
               markersize=8, alpha=0.7, linewidth=2)
        ax.plot(cum_pop, result['cum_freq2'], 
               '-o', color=result['cor2'], label=result['corpus2'],
               markersize=8, alpha=0.7, linewidth=2)
        
        # Encontrar ponto de máxima diferença
        diff = np.abs(result['cum_freq1'] - result['cum_freq2'])
        max_diff_idx = np.argmax(diff)
        
        # Plotar linha vertical KS
        ks_x = cum_pop[max_diff_idx]
        ks_y1 = result['cum_freq1'][max_diff_idx]
        ks_y2 = result['cum_freq2'][max_diff_idx]
        ax.vlines(x=ks_x, ymin=min(ks_y1, ks_y2), ymax=max(ks_y1, ks_y2),
                 color='black', linewidth=2, linestyle='--',
                 label='Distância KS')
        
        # Adicionar texto KS
        ax.text(ks_x + 2, np.mean([ks_y1, ks_y2]),
               f'Distância KS: {result["estatistica"]:.5%}\np-valor: {result["p_valor"]:.5f}',
               verticalalignment='center',
               bbox=dict(facecolor='white', edgecolor='gray', alpha=0.8))
        
        # Configurar gráfico
        ax.set_title(f'Teste de Kolmogorov-Smirnov:\n'
                    f'Comparação entre {result["corpus1"]} e {result["corpus2"]}',
                    fontsize=14, pad=20)
        ax.set_xlabel('População Cumulativa (%)', fontsize=12)
        ax.set_ylabel('Frequência Cumulativa (%)', fontsize=12)
        
        # Configurar eixos
        ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{int(x)}%'))
        ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{int(x)}%'))
        ax.yaxis.set_major_locator(MultipleLocator(5))
        ax.grid(True, linestyle='--', alpha=0.3)
        ax.set_xlim(-5, 105)
        
        # Adicionar interpretação
        interpretacao = "Interpretação do Teste:\n"
        if result['p_valor'] < 0.05:
            interpretacao += "Distribuições significativamente diferentes"
        else:
            interpretacao += "Distribuições estatisticamente similares"
        
        ax.text(0.02, 0.02, interpretacao,
                transform=ax.transAxes,
                bbox=dict(facecolor='white', edgecolor='gray', alpha=0.8))
        
        # Ajustar legenda e layout
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left',
                 title='Corpora Comparados')
        plt.tight_layout()
        
        # Salvar gráfico
        caminho = f"{caminho_grafico[:-4]}_{idx+1}.png"
        plt.savefig(caminho, dpi=300, bbox_inches='tight')
        plt.close()
    
    # Gerar relatório LaTeX
    arquivo_latex.write(
        "\\section{Teste de Kolmogorov-Smirnov (KS)}\n\n"
        
        "\\subsection{Introdução ao Teste KS}\n"
        "O teste de Kolmogorov-Smirnov é um teste estatístico não-paramétrico que avalia se duas amostras "
        "provêm da mesma distribuição. Este teste é particularmente útil para comparar distribuições de "
        "frequência de classes gramaticais entre diferentes corpora.\n\n"
    )
    
    # Tabela de resultados - Versão corrigida
    arquivo_latex.write(
        "\\subsection{Resultados das Comparações}\n"
        "\\begin{table}[H]\n"
        "\\small\n"  # Reduz o tamanho da fonte
        "\\setlength{\\tabcolsep}{5pt}\n"  # Ajusta o espaçamento entre colunas
        "\\centering\n"
        "\\begin{tabular}{|l|c|c|l|}\n\\hline\n"  # 'l' para alinhar texto à esquerda
        "\\textbf{Comparação} & "
        "\\textbf{\\begin{tabular}[c]{@{}c@{}}Estatística\\\\KS\\end{tabular}} & "
        "\\textbf{P-valor} & "
        "\\textbf{Interpretação} \\\\ \\hline\n"
    )
    
    for result in ks_results:
        interpretacao = "Significativamente diferentes" if result['p_valor'] < 0.05 else "Estatisticamente similares"
        arquivo_latex.write(
            f"{result['corpus1']} vs {result['corpus2']} & "
            f"{result['estatistica']:.4f} & "
            f"{result['p_valor']:.5f} & "
            f"{interpretacao} \\\\ \\hline\n"
        )
    
    arquivo_latex.write(
        "\\end{tabular}\n"
        "\\caption{Resultados dos Testes de Kolmogorov-Smirnov}\n"
        "\\end{table}\n\n"
    )
    
    # Adicionar gráficos
    for idx in range(len(ks_results)):
        caminho = f"{caminho_grafico[:-4]}_{idx+1}.png"
        arquivo_latex.write(salvar_grafico_e_gerar_latex(caminho, 
            f"Comparação KS: {ks_results[idx]['corpus1']} vs {ks_results[idx]['corpus2']}"))
    
    arquivo_latex.write("\\clearpage\n")
    
    return ks_results

In [7]:
def analise_diversidade_lexical(corpora_data, nomes_corpora, caminho_grafico_base, arquivo_latex):
    """
    Realiza uma análise comparativa da diversidade lexical entre múltiplos corpora (até 4).
    """
    if len(corpora_data) > 4:
        raise ValueError("Esta função suporta no máximo 4 corpora")
    
    if len(corpora_data) != len(nomes_corpora):
        raise ValueError("O número de corpora deve ser igual ao número de nomes")
    
    # Cores para cada corpus
    cores_pie = [
        ['skyblue', 'lightblue', 'powderblue'],
        ['lightcoral', 'indianred', 'rosybrown'],
        ['lightgreen', 'mediumseagreen', 'darkseagreen'],
        ['plum', 'orchid', 'thistle']
    ]
    cores_bar = ['skyblue', 'lightcoral', 'lightgreen', 'plum']
    
    def calcular_metricas(conllu_data):
        tokens = [token['form'].lower() for sentence in conllu_data for token in sentence]
        types = set(tokens)
        token_counter = Counter(tokens)
        total_tokens = len(tokens)
        total_types = len(types)
        hapax_legomena = sum(1 for freq in token_counter.values() if freq == 1)
        dis_legomena = sum(1 for freq in token_counter.values() if freq == 2)
        ttr = total_types / total_tokens
        return {
            'tokens': tokens,
            'types': types,
            'total_tokens': total_tokens,
            'total_types': total_types,
            'hapax': hapax_legomena,
            'dis': dis_legomena,
            'ttr': ttr,
            'counter': token_counter
        }
    
    # Calcular métricas para todos os corpora
    metricas = [calcular_metricas(corpus) for corpus in corpora_data]
    
    # Configurações gerais
    sns.set_style("whitegrid")
    plt.rcParams['font.size'] = 10
    
    # Gráfico 1: Comparação de métricas principais (gráficos de pizza)
    n_corpora = len(corpora_data)
    fig, axes = plt.subplots(1, n_corpora, figsize=(6*n_corpora, 6))
    if n_corpora == 1:
        axes = [axes]
    
    metrics = ['Hapax\nLegomena', 'Dis\nLegomena', 'Outros\nTypes']
    
    for i, (metrica, ax, nome) in enumerate(zip(metricas, axes, nomes_corpora)):
        values = [
            metrica['hapax']/metrica['total_tokens']*100,
            metrica['dis']/metrica['total_tokens']*100,
            (metrica['total_tokens'] - metrica['hapax'] - metrica['dis'])/metrica['total_tokens']*100
        ]
        
        wedges, _, _ = ax.pie(values, labels=None, autopct='', colors=cores_pie[i])
        ax.set_title(f'Distribuição - {nome}')
        
        # Legendas com porcentagens
        legend_labels = [f'{m} ({v:.1f}%)' for m, v in zip(metrics, values)]
        ax.legend(wedges, legend_labels, title="Métricas",
                 loc="center left", bbox_to_anchor=(1, 0, -0.2, 1))
    
    plt.tight_layout()
    plt.savefig(f"{caminho_grafico_base}_metricas.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # Gráfico 2: Type-Token Ratio
    plt.figure(figsize=(8, 6))
    ttr_data = [[nome, metrica['ttr']] for nome, metrica in zip(nomes_corpora, metricas)]
    sns.barplot(x=[d[0] for d in ttr_data], y=[d[1] for d in ttr_data], 
               palette=cores_bar[:len(corpora_data)])
    plt.title('Type-Token Ratio (TTR)')
    plt.ylabel('TTR')
    
    for i, (_, v) in enumerate(ttr_data):
        plt.text(i, v, f'{v:.3f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.savefig(f"{caminho_grafico_base}_ttr.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # Gerar relatório LaTeX
    arquivo_latex.write(
        "\\section{Análise Comparativa de Diversidade Lexical}\n"
        "Esta seção apresenta uma análise detalhada da diversidade lexical dos corpora, "
        "comparando diferentes métricas e distribuições entre os textos.\n\n"
        
        "\\subsection{Métricas Principais}\n"
        "As métricas a seguir nos permitem quantificar diferentes aspectos do uso do vocabulário:\n\n"
    )
    
    # Métricas para cada corpus
    for nome, metrica in zip(nomes_corpora, metricas):
        arquivo_latex.write(
            f"\\textbf{{{nome}}}:\n"
            "\\begin{itemize}\n"
            f"\\item Total de Tokens: {metrica['total_tokens']:,}\n"
            f"\\item Total de Types: {metrica['total_types']:,}\n"
            f"\\item Hapax Legomena: {metrica['hapax']:,} "
            f"({metrica['hapax']/metrica['total_tokens']*100:.1f}\\%)\n"
            f"\\item Dis Legomena: {metrica['dis']:,} "
            f"({metrica['dis']/metrica['total_tokens']*100:.1f}\\%)\n"
            f"\\item Type-Token Ratio (TTR): {metrica['ttr']:.3f}\n"
            "\\end{itemize}\n\n"
        )
    
    # Análise comparativa
    arquivo_latex.write(
        "\\subsection{Análise Comparativa}\n"
        "Comparando os corpora, observamos:\n"
        "\\begin{itemize}\n"
    )
    
    # Encontrar corpus com maior TTR
    max_ttr_idx = max(range(len(metricas)), key=lambda i: metricas[i]['ttr'])
    arquivo_latex.write(
        f"\\item {nomes_corpora[max_ttr_idx]} apresenta a maior diversidade lexical "
        f"(TTR = {metricas[max_ttr_idx]['ttr']:.3f})\n"
    )
    
    # Comparar tamanhos
    sizes = [f"{m['total_tokens']:,}" for m in metricas]
    arquivo_latex.write(
        f"\\item Tamanhos dos corpora: {', '.join(sizes)} tokens\n"
    )
    
    arquivo_latex.write("\\end{itemize}\n\n")
    
    # Análise dos gráficos
    arquivo_latex.write(
        "\\subsection{Análise dos Gráficos}\n\n"
        "\\textbf{1. Comparação de Métricas de Diversidade:}\n"
        "Os gráficos de pizza mostram a distribuição de:\n"
        "\\begin{itemize}\n"
        "\\item Hapax Legomena: Palavras que aparecem uma única vez\n"
        "\\item Dis Legomena: Palavras que aparecem duas vezes\n"
        "\\item Outras palavras únicas: Types que aparecem três ou mais vezes\n"
        "\\end{itemize}\n\n"
    )
    
    arquivo_latex.write(salvar_grafico_e_gerar_latex(f"{caminho_grafico_base}_metricas.png", 
        "Comparação de Métricas de Diversidade"))
    
    arquivo_latex.write(
        "\\textbf{2. Type-Token Ratio (TTR):}\n"
        "O gráfico de barras compara o TTR entre os corpora, onde valores mais altos "
        "indicam maior diversidade lexical.\n\n"
    )
    
    arquivo_latex.write(salvar_grafico_e_gerar_latex(f"{caminho_grafico_base}_ttr.png", 
        "Type-Token Ratio (TTR)"))
    
    arquivo_latex.write("\\clearpage\n")
    
    return metricas

In [8]:

# Função para salvar gráficos e gerar comandos LaTeX para inseri-los
def salvar_grafico_e_gerar_latex(caminho_grafico, titulo_grafico):
    return f"""
    \\begin{{figure}}[H]
        \\centering
        \\includegraphics[width=1.0\\textwidth]{{{caminho_grafico}}}
        \\caption{{{titulo_grafico}}}
    \\end{{figure}}
    """

def analise_frequencia_pos(corpora_data, nomes_corpora, caminho_grafico_base, arquivo_latex):
    """
    Análise de frequência de POS tags para múltiplos corpora (até 4)
    """
    if len(corpora_data) > 4:
        raise ValueError("Esta função suporta no máximo 4 corpora")
    
    if len(corpora_data) != len(nomes_corpora):
        raise ValueError("O número de corpora deve ser igual ao número de nomes")
    
    # Cores para cada corpus
    cores = ['skyblue', 'lightcoral', 'lightgreen', 'plum']
    
    # Coletar contagens para todos os corpora
    pos_counters = []
    total_tags = []
    for corpus in corpora_data:
        counter = Counter(token['upostag'] for sentence in corpus 
                        for token in sentence if token['upostag'] is not None)
        pos_counters.append(counter)
        total_tags.append(sum(counter.values()))
    
    # Obter todas as tags únicas
    pos_tags = sorted(set().union(*[set(counter.keys()) for counter in pos_counters]))
    
    # Calcular frequências e porcentagens
    frequencias = []
    porcentagens = []
    for counter, total in zip(pos_counters, total_tags):
        freq = [counter.get(tag, 0) for tag in pos_tags]
        frequencias.append(freq)
        porcentagens.append([(f / total) * 100 for f in freq])
    
    # Gráficos individuais de frequência
    for i, (freq, nome) in enumerate(zip(frequencias, nomes_corpora)):
        plt.figure(figsize=(10, 6))
        bars = plt.bar(pos_tags, freq, color=cores[i])
        
        # Adicionar valores em cima das barras
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(height):,}',
                    ha='center', va='bottom')
            
        plt.title(f'Distribuição de Frequência das Tags POS ({nome})')
        plt.xlabel('Tags POS')
        plt.ylabel('Frequência')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(f"{caminho_grafico_base}_{i+1}.png")
        plt.close()
    
    # Gráfico comparativo das porcentagens
    plt.figure(figsize=(12, 6))
    width = 0.8 / len(corpora_data)
    for i, (perc, nome, cor) in enumerate(zip(porcentagens, nomes_corpora, cores)):
        pos = [j + width * (i - (len(corpora_data)-1)/2) for j in range(len(pos_tags))]
        plt.bar(pos, perc, width=width, label=nome, color=cor)
    
    plt.title('Comparação de Porcentagens das Tags POS')
    plt.xlabel('Tags POS')
    plt.ylabel('Porcentagem (%)')
    plt.xticks(range(len(pos_tags)), pos_tags, rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{caminho_grafico_base}_comparativo.png")
    plt.close()
    
    # Gerar relatório LaTeX
    arquivo_latex.write("\\newpage\n\\section{Análise de Frequência das Tags POS}\n\n")
    
    arquivo_latex.write(
        "As etiquetas POS (Part-of-Speech) são rótulos que identificam as características "
        "linguísticas de cada palavra em um texto. Esta análise compara a distribuição "
        "dessas classes gramaticais entre os corpora.\n\n"
    )
    
    # Índice de Tags POS
    arquivo_latex.write(
        "\\subsection*{Índice de Tags POS}\n\n"
        "As principais tags utilizadas na análise são:\n\n"
        "\\begin{itemize}\n"
        "    \\item \\textbf{ADJ} - Adjetivos\n"
        "    \\item \\textbf{ADP} - Preposições\n" 
        "    \\item \\textbf{ADV} - Advérbios\n"
        "    \\item \\textbf{AUX} - Verbos Auxiliares\n"
        "    \\item \\textbf{CCONJ} - Conjunções Coordenativas\n"
        "    \\item \\textbf{DET} - Determinantes\n"
        "    \\item \\textbf{INTJ} - Interjeições\n"
        "    \\item \\textbf{NOUN} - Substantivos\n"
        "    \\item \\textbf{NUM} - Numerais\n"
        "    \\item \\textbf{PART} - Partículas\n"
        "    \\item \\textbf{PRON} - Pronomes\n"
        "    \\item \\textbf{PROPN} - Nomes Próprios\n"
        "    \\item \\textbf{PUNCT} - Pontuação\n"
        "    \\item \\textbf{SCONJ} - Conjunções Subordinativas\n"
        "    \\item \\textbf{SYM} - Símbolos\n"
        "    \\item \\textbf{VERB} - Verbos\n"
        "    \\item \\textbf{X} - Outros\n"
        "    \\item \\textbf{\\_} - Não Classificado\n"
        "\\end{itemize}\n\n"
    )
    
    # Tabela de Frequências
    arquivo_latex.write(
        "\\begin{table}[H]\n"
        "\\scriptsize\n"  # Fonte ainda menor que \small
        "\\setlength{\\tabcolsep}{3pt}\n"  # Espaçamento ainda menor entre colunas
        "\\centering\n"
        "\\begin{tabular}{|l|" + "r|"*len(corpora_data) + "}\\hline\n"
        "\\textbf{Tag} & " +  # Encurtando o cabeçalho
        " & ".join([f"\\textbf{{{nome}}}" for nome in nomes_corpora]) +  # Remove "Freq." do cabeçalho
        " \\\\ \\hline\n"
    )
    
    for i, tag in enumerate(pos_tags):
        freqs = [f"{int(freq[i]):,}" for freq in frequencias]
        arquivo_latex.write(f"{tag} & {' & '.join(freqs)} \\\\ \\hline\n")
    
    arquivo_latex.write(
        "\\end{tabular}\n"
        "\\caption{Frequência das Tags POS por Corpus}\n"
        "\\end{table}\n\n"
    )
    
    # Tabela de Porcentagens
    arquivo_latex.write("\\begin{table}[H]\n\\centering\n")
    arquivo_latex.write("\\begin{tabular}{|c|" + "c|"*len(corpora_data) + "}\\hline\n")
    arquivo_latex.write("Tag POS & " + " & ".join([f"\\% {nome}" for nome in nomes_corpora]) + " \\\\ \\hline\n")
    
    for i, tag in enumerate(pos_tags):
        percs = [f"{perc[i]:.2f}\\%" for perc in porcentagens]
        arquivo_latex.write(f"{tag} & {' & '.join(percs)} \\\\ \\hline\n")
    
    arquivo_latex.write("\\end{tabular}\n\\caption{Porcentagem das Tags POS}\n\\end{table}\n\n")
    
    # Inserir gráficos
    for i, nome in enumerate(nomes_corpora):
        arquivo_latex.write(salvar_grafico_e_gerar_latex(
            f"{caminho_grafico_base}_{i+1}.png",
            f"Distribuição de Frequência das Tags POS ({nome})"
        ))
    
    arquivo_latex.write(salvar_grafico_e_gerar_latex(
        f"{caminho_grafico_base}_comparativo.png",
        "Comparação de Porcentagens das Tags POS"
    ))
    
    arquivo_latex.write("\\clearpage\n")
    
    return {
        'frequencias': frequencias,
        'porcentagens': porcentagens,
        'tags': pos_tags
    }

def analise_comprimento_tokens(corpora_data, nomes_corpora, caminho_grafico_base, arquivo_latex):
    """
    Análise de comprimento dos tokens para múltiplos corpora (até 4)
    """
    if len(corpora_data) > 4:
        raise ValueError("Esta função suporta no máximo 4 corpora")
    
    if len(corpora_data) != len(nomes_corpora):
        raise ValueError("O número de corpora deve ser igual ao número de nomes")
    
    # Cores para cada corpus
    cores = ['#4FB6D6', '#FF6B6B', '#90EE90', '#DDA0DD']
    
    # Coletar comprimentos dos tokens para todos os corpora
    token_lengths = []
    total_tokens = []
    for corpus in corpora_data:
        lengths = [len(token['form']) for sentence in corpus for token in sentence]
        token_lengths.append(lengths)
        total_tokens.append(len(lengths))
    
    # Obter todos os comprimentos únicos
    all_lengths = sorted(set().union(*[set(lengths) for lengths in token_lengths]))
    
    # Calcular frequências em porcentagem
    freq_percents = []
    for lengths, total in zip(token_lengths, total_tokens):
        freq_percent = [(lengths.count(length) / total) * 100 for length in all_lengths]
        freq_percents.append(freq_percent)
    
    # Estatísticas comparativas
    estatisticas = []
    for lengths in token_lengths:
        estatisticas.append({
            'media': np.mean(lengths),
            'mediana': np.median(lengths),
            'std': np.std(lengths)
        })
    
    # Adicionar estatísticas no LaTeX
    arquivo_latex.write("\\section{Análise de Comprimento dos Tokens}\n")
    arquivo_latex.write(
        "O comprimento dos tokens é uma medida importante na análise linguística, "
        "pois nos permite entender a distribuição do tamanho das palavras em um corpus. "
        "Esta análise pode revelar padrões no uso de palavras curtas versus longas, "
        "além de ajudar a identificar possíveis diferenças estilísticas entre os corpora.\\\\\\\\\n"
    )
    
    # Tabela de estatísticas
    arquivo_latex.write("\\begin{table}[H]\n\\centering\n")
    arquivo_latex.write("\\begin{tabular}{|c|" + "c|"*len(corpora_data) + "}\\hline\n")
    arquivo_latex.write("Métrica & " + " & ".join(nomes_corpora) + " \\\\ \\hline\n")
    
    metricas = ['Comprimento Médio', 'Mediana', 'Desvio Padrão']
    for metrica in metricas:
        if metrica == 'Comprimento Médio':
            valores = [f"{e['media']:.2f}" for e in estatisticas]
        elif metrica == 'Mediana':
            valores = [f"{e['mediana']:.2f}" for e in estatisticas]
        else:  # Desvio Padrão
            valores = [f"{e['std']:.2f}" for e in estatisticas]
        arquivo_latex.write(f"{metrica} & {' & '.join(valores)} \\\\ \\hline\n")
    
    arquivo_latex.write("\\end{tabular}\n\\caption{Comparação do Comprimento dos Tokens}\n\\end{table}\n\n")
    
    # Filtrar comprimentos até 30 para os gráficos
    max_length = 30
    filtered_lengths = [l for l in all_lengths if l <= max_length]
    filtered_freqs = []
    for freqs in freq_percents:
        filtered_freqs.append([f for l, f in zip(all_lengths, freqs) if l <= max_length])
    
    # Gráficos individuais
    for i, (freqs, nome, cor) in enumerate(zip(filtered_freqs, nomes_corpora, cores)):
        plt.figure(figsize=(12, 7))
        plt.bar(filtered_lengths, freqs, color=cor, alpha=0.8)
        plt.title(f'Distribuição do Comprimento dos Tokens ({nome})', fontsize=14, pad=20)
        plt.xlabel('Comprimento dos Tokens', fontsize=12)
        plt.ylabel('Frequência (%)', fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.xlim(-0.5, max_length + 0.5)
        plt.ylim(0, max(freqs) * 1.15)
        
        # Adiciona rótulos para valores significativos
        for j, v in enumerate(freqs):
            if v > 0.5:
                plt.text(filtered_lengths[j], v, f'{v:.1f}%', ha='center', va='bottom')
        
        plt.xticks(filtered_lengths)
        plt.tight_layout()
        plt.savefig(f"{caminho_grafico_base}_{i+1}.png", dpi=300, bbox_inches='tight')
        plt.close()
    
    # Gráfico comparativo
    plt.figure(figsize=(16, 8))
    width = 0.8 / len(corpora_data)
    x = np.arange(len(filtered_lengths))
    
    for i, (freqs, nome, cor) in enumerate(zip(filtered_freqs, nomes_corpora, cores)):
        offset = width * (i - (len(corpora_data)-1)/2)
        plt.bar(x + offset, freqs, width=width, label=nome, color=cor, alpha=0.8)
    
    plt.title('Comparação de Comprimento dos Tokens', fontsize=14, pad=20)
    plt.xlabel('Comprimento dos Tokens', fontsize=12)
    plt.ylabel('Frequência (%)', fontsize=12)
    plt.xlim(-0.8, max(x) + 0.8)
    plt.ylim(0, max([max(f) for f in filtered_freqs]) * 1.15)
    
    plt.xticks(x, filtered_lengths)
    plt.legend(fontsize=10, framealpha=0.9, loc='upper right')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f"{caminho_grafico_base}_comparativo.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # Inserir gráficos no LaTeX
    for i, nome in enumerate(nomes_corpora):
        arquivo_latex.write(salvar_grafico_e_gerar_latex(
            f"{caminho_grafico_base}_{i+1}.png",
            f"Distribuição de Comprimento dos Tokens ({nome})"
        ))
    
    arquivo_latex.write(salvar_grafico_e_gerar_latex(
        f"{caminho_grafico_base}_comparativo.png",
        "Comparação de Comprimento dos Tokens"
    ))
    
    arquivo_latex.write("\\clearpage\n")
    
    return {
        'estatisticas': estatisticas,
        'comprimentos': token_lengths,
        'frequencias': freq_percents
    }

def analise_frequencia_por_sentenca(corpora_data, nomes_corpora, caminho_grafico_base, arquivo_latex):
    """
    Análise de frequência por sentença para múltiplos corpora (até 4)
    """
    if len(corpora_data) > 4:
        raise ValueError("Esta função suporta no máximo 4 corpora")
    
    if len(corpora_data) != len(nomes_corpora):
        raise ValueError("O número de corpora deve ser igual ao número de nomes")
    
    n_corpora = len(corpora_data)
    cores = ['skyblue', 'lightcoral', 'lightgreen', 'plum']
    
    # Calcular tokens por sentença para todos os corpora
    tokens_por_sentenca = []
    stats = []
    
    for corpus in corpora_data:
        tokens = [len(sentence) for sentence in corpus]
        tokens_por_sentenca.append(tokens)
        
        stats.append({
            'média': np.mean(tokens),
            'mediana': np.median(tokens),
            'desvio': np.std(tokens),
            'mínimo': min(tokens),
            'máximo': max(tokens),
            'total_sentencas': len(tokens)
        })
    
    # Criar subplots
    fig, axes = plt.subplots(1, len(corpora_data), figsize=(6*len(corpora_data), 6))
    if len(corpora_data) == 1:
        axes = [axes]
    
    # Determinar bins fixos até 100
    bins = np.linspace(0, 100, 50)
    
    # Criar histogramas
    for i, (tokens, stat, nome, cor, ax) in enumerate(zip(tokens_por_sentenca, stats, nomes_corpora, cores, axes)):
        # Filtrar tokens para mostrar apenas até 100
        tokens_filtrados = [t for t in tokens if t <= 100]
        
        # Criar histograma com frequências relativas (em porcentagem)
        n, _, _ = ax.hist(tokens_filtrados, bins=bins, alpha=0.7, color=cor, label=nome,
                         weights=np.ones_like(tokens_filtrados) * 100 / len(tokens))
        
        # Adicionar linhas de média e mediana
        if stat['média'] <= 100:
            ax.axvline(stat['média'], color='blue', linestyle='dashed', 
                      label=f"Média ({stat['média']:.1f})")
        if stat['mediana'] <= 100:
            ax.axvline(stat['mediana'], color='darkblue', linestyle='dashed', 
                      label=f"Mediana ({stat['mediana']:.1f})")
        
        # Configurar rótulos e título
        ax.set_title(f'Distribuição - {nome}')
        ax.set_xlabel('Tokens por Sentença')
        ax.set_ylabel('Frequência (%)')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        # Fixar limites
        ax.set_xlim(0, 100)
        ax.set_ylim(0, 25)  # Ajuste este valor conforme necessário
    
    plt.tight_layout()
    plt.savefig(caminho_grafico_base, dpi=300, bbox_inches='tight')
    plt.close()
    
    
    # Gerar relatório LaTeX
    arquivo_latex.write(
        "\\section{Análise Comparativa do Tamanho das Sentenças}\n"
        "Esta seção apresenta uma análise detalhada da distribuição do número de tokens "
        "por sentença nos corpora. Esta métrica nos ajuda a entender a complexidade "
        "estrutural das sentenças e as diferenças de estilo entre os textos.\n\n"
    )
    
        # Tabela de estatísticas - Versão corrigida
    arquivo_latex.write(
        "\\subsection{Estatísticas Descritivas}\n"
        "\\begin{table}[H]\n"
        "\\small\n"  # Reduz o tamanho da fonte
        "\\setlength{\\tabcolsep}{5pt}\n"  # Ajusta o espaçamento entre colunas
        "\\centering\n"
        "\\begin{tabular}{|l|" + "r|"*n_corpora + "}\\hline\n"  # 'r' para alinhar números à direita
        "\\textbf{Métrica} & " + " & ".join([f"\\textbf{{{nome}}}" for nome in nomes_corpora]) + 
        " \\\\ \\hline\n"
    )
    
    metricas = [
        ('Número de Sentenças', 'total_sentencas', '{:,}'),
        ('Média de Tokens', 'média', '{:.2f}'),
        ('Mediana', 'mediana', '{:.2f}'),
        ('Desvio Padrão', 'desvio', '{:.2f}'),
        ('Mínimo', 'mínimo', '{:d}'),
        ('Máximo', 'máximo', '{:d}')
    ]
    
    for nome_metrica, chave, formato in metricas:
        valores = [formato.format(stat[chave]) for stat in stats]
        arquivo_latex.write(f"\\textbf{{{nome_metrica}}} & {' & '.join(valores)} \\\\ \\hline\n")
    
    arquivo_latex.write(
        "\\end{tabular}\n"
        "\\caption{Estatísticas do Tamanho das Sentenças}\n"
        "\\end{table}\n\n"
    )
    
    # Análise comparativa
    arquivo_latex.write("\\subsection{Análise Comparativa}\n")
    
    # Encontrar corpus com maior média
    max_media_idx = max(range(len(stats)), key=lambda i: stats[i]['média'])
    arquivo_latex.write(
        f"{nomes_corpora[max_media_idx]} apresenta, em média, as sentenças mais longas "
        f"({stats[max_media_idx]['média']:.1f} tokens por sentença).\n\n"
    )
    
    # Encontrar corpus com maior variabilidade
    max_desvio_idx = max(range(len(stats)), key=lambda i: stats[i]['desvio'])
    arquivo_latex.write(
        f"{nomes_corpora[max_desvio_idx]} mostra maior variabilidade no tamanho das sentenças, "
        f"com desvio padrão de {stats[max_desvio_idx]['desvio']:.1f} tokens.\n\n"
    )
    
    arquivo_latex.write(
        "\\subsection{Interpretação dos Histogramas}\n"
        "Os histogramas mostram a distribuição do número de tokens por sentença em cada corpus. "
        "As linhas verticais indicam a média e a mediana, permitindo visualizar:\n"
        "\\begin{itemize}\n"
        "\\item A forma geral da distribuição e sua simetria\n"
        "\\item A concentração de sentenças em determinados tamanhos\n"
        "\\item A presença de outliers (sentenças muito curtas ou muito longas)\n"
        "\\item A relação entre média e mediana, indicando possível assimetria\n"
        "\\end{itemize}\n\n"
    )
    
    # Inserir gráfico
    arquivo_latex.write(salvar_grafico_e_gerar_latex(caminho_grafico_base, 
        "Distribuição Comparativa do Tamanho das Sentenças"))
    
    arquivo_latex.write("\\clearpage\n")
    
    return {
        'estatisticas': stats,
        'tokens_por_sentenca': tokens_por_sentenca  # Corrigido aqui
    }

def gerar_relatorio_latex(corpora_data, nomes_corpora, arquivo_saida_latex):
    """
    Gera um relatório LaTeX comparativo para múltiplos corpora
    
    Args:
        corpora_data: Lista com os dados CONLLU dos corpora
        nomes_corpora: Lista com os nomes dos corpora
        arquivo_saida_latex: Caminho do arquivo LaTeX de saída
    """
    # Caminhos base para os gráficos
    caminhos_graficos = {
        'pos': "distribuicao_frequencia_tags",
        'comprimento': "distribuicao_comprimento_tokens",
        'sentenca': "distribuicao_tags_sentenca",
        'jensen_shannon': "distribuicao_jensen_shannon",
        'diversidade': "diversidade_lexical",
        'ks': "distribuicao_ks",
        'freq_sentenca': "distribuicao_frequencia_por_sentenca"
    }

    with open(arquivo_saida_latex, 'w', encoding='utf-8') as arquivo_latex:
        # Cabeçalho do arquivo LaTeX
        arquivo_latex.write(
            "\\documentclass{article}\n"
            "\\usepackage[utf8]{inputenc}\n"
            "\\usepackage[T1]{fontenc}\n"
            "\\usepackage[brazil]{babel}\n"
            "\\usepackage{graphicx}\n"
            "\\usepackage{float}\n"
            "\\usepackage{array}\n"
            "\\begin{document}\n"
        )
        
        # Título
        titulo = "Análise Estatística Comparativa:\\\\%s" % " vs. ".join(nomes_corpora)
        arquivo_latex.write(f"\\title{{{titulo}}}\n\\maketitle\n")
        
        # Análises
        analise_frequencia_pos(
            corpora_data, 
            nomes_corpora, 
            caminhos_graficos['pos'],
            arquivo_latex
        )
        
        analise_comprimento_tokens(
            corpora_data,
            nomes_corpora,
            caminhos_graficos['comprimento'],
            arquivo_latex
        )
        
        analise_tags_por_sentenca(
            corpora_data,
            nomes_corpora,
            caminhos_graficos['sentenca'],
            arquivo_latex
        )
        
        analise_diversidade_lexical(
            corpora_data,
            nomes_corpora,
            caminhos_graficos['diversidade'],
            arquivo_latex
        )
        
        analise_jensen_shannon(
            corpora_data,
            nomes_corpora,
            caminhos_graficos['jensen_shannon'],
            arquivo_latex
        )
        
        teste_ks(
            corpora_data,
            nomes_corpora,
            caminhos_graficos['ks'],
            arquivo_latex
        )
        
        analise_frequencia_por_sentenca(
            corpora_data,
            nomes_corpora,
            caminhos_graficos['freq_sentenca'],
            arquivo_latex
        )
        
        arquivo_latex.write("\\end{document}")

    print(f"Relatório LaTeX salvo como {arquivo_saida_latex}.")

# Uso da função com os três corpora
corpora_data = [dados_conllu1, dados_conllu2, dados_conllu3]
nomes_corpora = [Dantes_Manual, Dantes_Automatizado, Dantes_Large]

gerar_relatorio_latex(corpora_data, nomes_corpora, "relatorio_comparativo.tex")

C:\Users\Marcos\AppData\Local\Temp\ipykernel_23692\2913429422.py:77: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=[d[0] for d in ttr_data], y=[d[1] for d in ttr_data],


Relatório LaTeX salvo como relatorio_comparativo.tex.
